In [2]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [3]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."


import tensorboard
tensorboard.__version__

TensorFlow version:  2.16.1


'2.16.2'

In [4]:
import pandas as pd
import ast

data_path = '../paired_data.csv'

data = pd.read_csv(data_path)
data["overview_embedding"] = data["overview_embedding"].apply(ast.literal_eval)

In [5]:
from sentence_transformers import SentenceTransformer
transformer = SentenceTransformer('all-MiniLM-L6-v2')

/home/matt/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/matt/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [28]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from joblib import dump

# Convert embedding column from list to numpy array
data["overview_embedding"] = data["overview_embedding"].apply(np.array)

# Split the data into features (X) and target (y)
X = np.vstack(data["overview_embedding"].values)
y = data['average_rating']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

# Standardize features
x_scaler = StandardScaler()
X_train_scaled = x_scaler.fit_transform(X_train)
X_test_scaled = x_scaler.transform(X_test)

# Define the neural network architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1)
])

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss_fn = tf.keras.losses.MeanSquaredError(name="mean_squared_error")
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Checkpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the model
model.fit(X_train_scaled, y_train, epochs=30, validation_split=0.1, callbacks=[checkpoint_callback, tensorboard_callback])

# Evaluate the model
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

# Save the model
dump(x_scaler, 'scaler.joblib')
model.save('movie_rating_prediction_model.keras')

Epoch 1/30


/home/matt/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/168 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0028 - loss: 7340.3525
Epoch 1: val_loss improved from inf to 0.69160, saving model to best_model.keras
168/168 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.0028 - loss: 7272.1372 - val_accuracy: 0.0101 - val_loss: 0.6916
Epoch 2/30
162/168 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0050 - loss: 46.4386
Epoch 2: val_loss improved from 0.69160 to 0.44096, saving model to best_model.keras
168/168 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.0050 - loss: 45.4284 - val_accuracy: 0.0101 - val_loss: 0.4410
Epoch 3/30
164/168 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0057 - loss: 14.4853
Epoch 3: val_loss did not improve from 0.44096
168/168 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.0057 - loss: 15.3190 - val_accuracy: 0.0000e+00 - val_loss: 17.3996
Epoch 4/30
163/168 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0020 - loss: 11.5097
Epoch 4: val_loss improved from 0.44096 to 0.43240, saving model to best_model.keras
1

In [24]:
from sklearn.preprocessing import StandardScaler
from joblib import load
import numpy as np

overview: str = input("Give your movie overview: ")
print(overview)

# Now you can use the loaded scaler to transform new data
scaled_embed = x_scaler.transform(np.array(transformer.encode(overview)).reshape(1, -1))

predicted_rating = model.predict(scaled_embed)

print(f"prediction: {predicted_rating}")

The filmmaking team behind the hits "Scary Movie," "Date Movie," "Epic Movie" and "Meet The Spartans" this time puts its unique, inimitable stamp on one of the biggest and most bloated movie genres of all time -- the disaster film.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
prediction: [[3.2491636]]
